In [1]:
import pandas as pd
import numpy as np

#Import dataset and extarct necessary columns for the testing

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/Amazon-Musical-Reviews-Rating-Dataset/master/Musical_instruments_reviews.csv' , usecols=['reviewText', 'overall'])

In [9]:
df.head()

,reviewText,overall
0,"Not much to write about here, but it does exac...",5.0
1,The product does exactly as it should and is q...,5.0
2,The primary job of this device is to block the...,5.0
3,Nice windscreen protects my MXL mic and preven...,5.0
4,This pop filter is great. It looks and perform...,5.0


In [10]:
df['overall'].value_counts()

5.0    6938
4.0    2084
3.0     772
2.0     250
1.0     217
Name: overall, dtype: int64

#Install packages for the preprocess cleaning

In [3]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-b9tfibyg
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-b9tfibyg
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-cp37-none-any.whl size=11759 sha256=2d1c3ea86dd3474dacbe798ffb5db917ceb9cda3333783ba08bf3392bfa26615
  Stored in directory: /tmp/pip-ephem-wheel-cache-2ybqja6o/wheels/a8/18/22/90afa4bd43247fb9a75b710a4a3fcd94966c022ce9e3c7d0a6
Successfully built preprocess-kgptalkie


In [11]:
import preprocess_kgptalkie as ps
import re

In [12]:
#cleaning function
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [14]:
#apply cleaning function
df['reviewText'] = df['reviewText'].apply(lambda x: get_clean(x))

In [15]:
df.head()

,reviewText,overall
0,not much to write about here but it does exact...,5.0
1,the product does exactly as it should and is q...,5.0
2,the primary job of this device is to block the...,5.0
3,nice windscreen protects my mxl mic and preven...,5.0
4,this pop filter is great it looks and performs...,5.0


#TFIDF and Linear SVM Algorithm


In [16]:
#libraries for the algorithm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [39]:
#apply algorithm for the testing
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,5), analyzer='char')

In [41]:
X = tfidf.fit_transform(df['reviewText'])
Y = df['overall']

In [42]:
X.shape, Y.shape

((10261, 20000), (10261,))

In [43]:
#build values and parameters
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.8, random_state = 0)

In [44]:
X_train.shape

(2052, 20000)

In [54]:
#the linear SVM model
clf = LinearSVC( C = 20, class_weight="balanced")
clf.fit(X_train, Y_train)

LinearSVC(C=20, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [55]:
#predictions for the test cases in the dataset
y_pred = clf.predict(X_test)

In [56]:
#classification report summary
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.29      0.04      0.07       176
         2.0       0.09      0.01      0.02       207
         3.0       0.24      0.15      0.18       609
         4.0       0.30      0.29      0.29      1655
         5.0       0.75      0.82      0.78      5562

    accuracy                           0.63      8209
   macro avg       0.33      0.26      0.27      8209
weighted avg       0.59      0.63      0.61      8209



#Testing for the real data from the source of example

In [61]:
#values-01
x = 'this is reaclly bad.'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

array([5.])

In [59]:
#values-02
x = 'this is a really nice place for visit'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

array([5.])

In [1]:
#values-03
x = 'it is not that good'
x = get_clean(x)
vec = tfidf.transform([x])
clf.predict(vec)

NameError: name 'get_clean' is not defined